# Compare behavior cloning

This notebook allows the visual comparison of various behavior cloning controllers, as shown in their verify runs. The notebook starts with an experiment that lists several bc verify type experiments. It is assumed that those experiments had been run before this notebook is running, because it is building on the results of those experiments. 

In [ ]:
import sys
sys.path.append("..")

from exp_run_config import Config
Config.PROJECTNAME = "BerryPicker"

import pathlib
import pprint
import matplotlib.pyplot as plt
# fixing the fact that it defaults to Type 3 fonts
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

import numpy as np

In [ ]:
experiment = "behavior_cloning"
run = "bc_compare_00"

exp = Config().get_experiment(experiment, run)
pprint.pprint(exp)

verification_run_exps = {}
for expname in exp["bc_verification_runs"]:
    verexp = Config().get_experiment(experiment, expname)
    verification_run_exps[expname] = verexp

In [ ]:
# load the saved data from the verifications
real_np = None
preds_nps = {}
for name in verification_run_exps:
    verexp = verification_run_exps[name]
    if real_np is None:
        real_path = pathlib.Path(verexp.data_dir(), "real.npy")
        real_np = np.load(real_path)
    pred_path =  pathlib.Path(verexp.data_dir(), "pred.npy")
    pred_np = np.load(pred_path)
    preds_nps[name] = pred_np

In [ ]:
fig, axs = plt.subplots(2,3, figsize=(8, 6))
titles = ["height","distance", "heading", "wrist_angle", "wrist_rotation", "gripper"]
# for i in range(bcexp["control_size"]):
for i in range(6):
    ax = axs[i//3, i%3]
    # ax.set_ylim(0, 1.4)
    for name in preds_nps:
        ax.plot(preds_nps[name][:,i], label=name)
    ax.plot(real_np[:,i], label="ground truth")
    if i==0:
        handles, labels = ax.get_legend_handles_labels()
        fig.legend(handles, labels, ncol=1+len(verification_run_exps),
            bbox_to_anchor=(0.5, 0), loc="upper center")
    ax.set_title(titles[i])

plt.tight_layout()
graphfilename = pathlib.Path(exp.data_dir(), "bccomparison.pdf")
plt.savefig(graphfilename, bbox_inches='tight')